# Media GeoTag Mapper (MGTM): iPhone Media Example 

By Kenneth Burchfiel

Released under the MIT license

GitHub link: https://github.com/kburchfiel/media_geotag_mapper

This notebook allows me to test out Media GeoTag Mapper functions on iPhone pictures and clips. The steps are identical to those in the main tutorial notebook, so the documentation within this particular notebook is very limited.

Since I don't own an iPhone, I tested out this notebook on iPhone image and video files that my wife had taken. I won't be sharing the maps or other output files here, but those files are very similar to the ones found in the main tutorial notebook.


In [1]:
import time
program_start_time = time.time() # Allows the program's runtime to be measured
import os
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
import IPython.display
from IPython.display import display

from media_geotag_functions_v6 import generate_media_list, \
    retrieve_pic_locations, retrieve_clip_locations, generate_loc_list, \
    map_media_locations, folder_list_to_map, flip_lon, create_map_screenshot, \
    calculate_distance_by_year, convert_png_to_smaller_jpg, \
    batch_create_map_screenshots, batch_convert_pngs_to_smaller_jpgs

In [22]:
connection_type = 'sftp' # Can be 'sftp' or 'local'; this was added 
# for my own convenience but may not be necessary in your use case.

if connection_type == 'sftp':
    top_folder = '/home/kjb3/sshfs_ext_rclone_mount/'
else:
    top_folder = '/media/kjb3/KJB320TB1/'

folder_name = 'combined_iphone'
path_to_map_folder = os.getcwd()+'/iphone_maps'
screenshot_save_path = 'iphone_map_screenshots'


top_folder_list = [
f'{top_folder}D1V1/Vids Clips Pics/1 Unsorted Pics/41 Allie Cameras',
f'{top_folder}D2V1/Vids Clips Pics/2 Unsorted Clips/41 Allie Cameras'
]

top_folder_list

['/home/kjb3/sshfs_ext_rclone_mount/D1V1/Vids Clips Pics/1 Unsorted Pics/41 Allie Cameras',
 '/home/kjb3/sshfs_ext_rclone_mount/D2V1/Vids Clips Pics/2 Unsorted Clips/41 Allie Cameras']

In [3]:
generate_new_lists = False

In [23]:
if generate_new_lists == True:
    df_media = generate_media_list(top_folder_list=top_folder_list,
    folder_name = folder_name, files_to_import = 0)

In [24]:
df_media = pd.read_csv(f'{folder_name}_media_list.csv')
# df_media.head(5)

I'll next create df_all_locations, a DataFrame containing all location data provided for the files in df_media. I'll limit the sample output of this notebook to geotags taken within Colorado.

In [25]:
if generate_new_lists == True:
    df_all_locations = generate_loc_list(df_media = df_media, 
    folder_name = folder_name)
    df_all_locations = flip_lon(df_all_locations, lat_south_bound = 25,
    lat_north_bound = 45, lon_west_bound = 70, lon_east_bound = 95)
    df_all_locations.to_csv(f'{folder_name}_media_locations.csv', index = False)

In [26]:
df_all_locations = pd.read_csv(
    f'{folder_name}_media_locations.csv').reset_index(drop=True)
# Converting date/time columns from the .csv file back into UTC DateTimes:
for col in ['utc_ctime_estimate',
            'utc_modified_time_estimate',
            'utc_accessed_time_estimate',
            'utc_metadata_creation_time']:
    df_all_locations[col] = pd.to_datetime(df_all_locations[col], utc=True,
                                          format='mixed')


The following code modifies df_all_locations to filter out media whose EXIF data or metadata didn't have an utc_metadata_creation_time value. It also sorts df_all_locations by utc_metadata_creation_time and removes rows that lacked geotag data. These steps are taken to prepare the dataset for mapping tasks.

In [27]:
print(len(df_all_locations))
df_locations = df_all_locations.query("lat != 0 & lon != 0").copy()
print(len(df_locations))
df_locations.query("utc_metadata_creation_time.notna()", inplace = True)
print(len(df_locations))
df_locations.reset_index(drop=True,inplace=True)
df_locations.sort_values('utc_metadata_creation_time', inplace = True)
df_locations.to_csv('combined_iphone_locations_with_valid_dates.csv', index = False)

783
751
751


In [9]:
combined_map = map_media_locations(df_locations, folder_path = path_to_map_folder, 
timestamp_column = 'utc_metadata_creation_time',
file_name = 'combined_iphone', zoom_start = 6)

Added 751 markers to the map.


In [10]:
create_map_screenshot(path_to_map_folder = 
path_to_map_folder, map_name= 'iphone_combined_locations.html', 
screenshot_save_path = screenshot_save_path)
# IPython.display.display(IPython.display.Image(
#     filename='smaller_screenshots/'+'combined_locations.jpg'))

In [11]:
map_media_locations(df_locations, folder_path = path_to_map_folder, 
file_name = 'combined_routes', timestamp_column = 'utc_metadata_creation_time', 
add_paths = True, zoom_start = 6)
print("Done")

Added 751 markers to the map.
Done


In [12]:
create_map_screenshot(path_to_map_folder = path_to_map_folder,
map_name= 'combined_routes_locations.html', 
screenshot_save_path = screenshot_save_path)
# IPython.display.display(IPython.display.Image(
#     filename='smaller_screenshots/'+'combined_routes_locations.jpg'))

In [13]:
create_map_screenshot(path_to_map_folder = path_to_map_folder,
map_name= 'combined_routes_intl_locations.html', 
screenshot_save_path = screenshot_save_path)
# IPython.display.display(IPython.display.Image(
#     filename='smaller_screenshots/'+'combined_routes_intl_locations.jpg'))

In [14]:
international_travel_years = [2024, 2025]

map_dict = {}
for i in range(2018, datetime.date.today().year+1):
    # Determining the start times (in UTC) of the current year 
    # and the following year: (We need UTC-based times in order to 
    # accommodate the UTC-based times within utc_metadata_creation_time.)
    start_time = f"{str(i)}-01-01 00:00+00:00"
    next_year_start_time = f"{str(i+1)}-01-01 00:00+00:00"
    df_locs_for_year = df_locations.query(
        "utc_metadata_creation_time >= @start_time \
& utc_metadata_creation_time < @next_year_start_time")
    if len(df_locs_for_year) > 0:
        print(f"Creating map for {i}:")
        map_dict[i] = map_media_locations(df_locs_for_year,
            folder_path = path_to_map_folder, file_name = f'{i}_combined', 
            add_paths = True, zoom_start = 6)
        if i in international_travel_years:
            map_media_locations(df_locations.query(
                "utc_metadata_creation_time >= @start_time \
    & utc_metadata_creation_time < @next_year_start_time"),
                folder_path = path_to_map_folder, file_name = f'{i}_combined_intl',
                add_paths = True, zoom_start = 4)

Creating map for 2025:
Added 727 markers to the map.
Added 727 markers to the map.
Creating map for 2026:
Added 24 markers to the map.


In [15]:
batch_create_map_screenshots(path_to_map_folder = 
path_to_map_folder, screenshot_save_path = 
screenshot_save_path)

In [16]:
df_distances_by_year = calculate_distance_by_year(
    df_locations)

#df_distances_by_year

In [17]:
# sum(df_distances_by_year['total_distance'])

In [18]:
program_end_time = time.time()
run_time = program_end_time - program_start_time
run_minutes = run_time // 60
run_seconds = run_time % 60
print("Completed run at",time.ctime(program_end_time),"(local time)")
print("Total run time:",'{:.2f}'.format(run_time),
"second(s) ("+str(run_minutes),"minute(s) and",'{:.2f}'.format(run_seconds),
"second(s))") 

Completed run at Mon Jan  5 23:50:15 2026 (local time)
Total run time: 51.56 second(s) (0.0 minute(s) and 51.56 second(s))
